In [ ]:
import utils
import models.categorical
import pandas as pd
from datetime import date

from sklearn.svm import SVC
from sklearn.model_selection import ParameterGrid
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from progressbar import ProgressBar

import warnings
warnings.filterwarnings("ignore")

# Hyperparameter Tuning

In [ ]:
futuresList = utils.futuresList

svmParams = [
    {
        "C": [0.1, 0.5, 1.0, 1.5],
        "kernel": ["poly", "rbf", "sigmoid"],
        "gamma": ["scale", "auto"],
        "class_weight": ["balanced", None] 
    }
]

start_date = date(2000, 1, 1)

In [ ]:
# retrieve parameter grid
pbar = ProgressBar()

parameter_grid = list(ParameterGrid(svmParams))

for future in pbar(futuresList):
    print(future)
    # load data
    df = pd.read_csv(f"tickerData/{future}.txt", parse_dates = ["DATE"])
    df.columns = ['DATE', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL', 'OI', 'P', 'R', 'RINFO']
    df = df.set_index("DATE")
    df = df[(df.VOL != 0) & (df.CLOSE != 0)]
    df = df.dropna(axis=0)
    
    # load X and y
    X_df = utils.generate_X_df([df.CLOSE, df.VOL], ["perc", "perc"])
    y_df = utils.generate_y_cat(df.CLOSE)
    cost_df = utils.perc_change(df.CLOSE, shift=0)
    
    # prepare collated results
    agg_results_collated = pd.DataFrame(index=list(range(len(parameter_grid))), 
                                        columns=["accuracy_SMA", "opp_cost_SMA", "accuracy_EMA", "opp_cost_EMA"])
    win_results_collated = []

    # run walk forward validation 
    for i in range(len(parameter_grid)):
        param_set = parameter_grid[i]
        model = make_pipeline(StandardScaler(), SVC(**param_set))
        win_results, agg_results = models.categorical.walk_forward(
            model = model, X = X_df, y = y_df, cost_weight = cost_df, rolling = True, 
            max_windows = 100, start_index = start_date
        )
        win_results_collated.append(win_results)
        agg_results_collated.loc[i, "accuracy_SMA"] = agg_results.loc["SMA", "accuracy"]
        agg_results_collated.loc[i, "accuracy_EMA"] = agg_results.loc["EMA", "accuracy"]
        agg_results_collated.loc[i, "opp_cost_SMA"] = agg_results.loc["SMA", "opp_cost"]
        agg_results_collated.loc[i, "opp_cost_EMA"] = agg_results.loc["EMA", "opp_cost"]

    # save parameters
    parameter_df = pd.DataFrame.from_records(parameter_grid)
    combined_df = pd.concat([parameter_df, agg_results_collated], axis=1)
    combined_df.to_csv(f"model_metrics/svm/perc/{future}.csv", index=False)

# Save Models

In [ ]:
import utils
import pickle
import numpy as np
import pandas as pd
import models.categorical
from sklearn.svm import SVC

import warnings
warnings.filterwarnings("ignore")

train_start = '2019-01-01'
train_end = '2020-12-31'

In [ ]:
futuresList = utils.futuresList

for future in futuresList:
    txt_path = f'svm/perc/{future}'

    # load data
    df = pd.read_csv(f"tickerData/{future}.txt", parse_dates = ["DATE"])
    df.columns = ['DATE', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL', 'OI', 'P', 'R', 'RINFO']
    # extract train set
    train_df = df.loc[(df['DATE'] >= train_start) & (df['DATE'] <= train_end)]
    train_df = train_df.set_index("DATE")
    # remove na and zero values
    train_df = train_df[(train_df.VOL != 0) & (train_df.CLOSE != 0)]
    train_df = train_df.dropna(axis=0)

    # extract X_train and y_train
    X_train = utils.generate_X_df([train_df.CLOSE, train_df.VOL], ["perc", "perc"])
    y_train = utils.generate_y_cat(train_df.CLOSE)
    X_train = X_train.dropna(axis=0)
    y_train = y_train.dropna(axis=0)

    # get intersection of all dataframes
    common_index = X_train.index.intersection(y_train.index)
    X_train = X_train[X_train.index.isin(common_index)].to_numpy()
    y_train = y_train[y_train.index.isin(common_index)].to_numpy(np.dtype(int))
    
    model = SVC(C=1.5, kernel="rbf", gamma="scale", class_weight="balanced", probability=True)
    fitted = model.fit(X_train, y_train)
    save = models.categorical.SVMWrapper(model=fitted)
    
    with open(f'saved_models/categorical/{txt_path}.p', 'wb') as f:
        pickle.dump(save, f)

# Validation Sharpe

In [ ]:
import utils
import pickle
import numpy as np
import pandas as pd
import models.categorical
from sklearn.svm import SVC

import warnings
warnings.filterwarnings("ignore")

train_start = '2018-09-01'
train_end = '2020-09-30'

In [ ]:
futuresList = utils.futuresList

for future in futuresList:
    txt_path = f'svm/perc/{future}'

    # load data
    df = pd.read_csv(f"tickerData/{future}.txt", parse_dates = ["DATE"])
    df.columns = ['DATE', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL', 'OI', 'P', 'R', 'RINFO']
    # extract train set
    train_df = df.loc[(df['DATE'] >= train_start) & (df['DATE'] <= train_end)]
    train_df = train_df.set_index("DATE")
    # remove na and zero values
    train_df = train_df[(train_df.VOL != 0) & (train_df.CLOSE != 0)]
    train_df = train_df.dropna(axis=0)

    # extract X_train and y_train
    X_train = utils.generate_X_df([train_df.CLOSE, train_df.VOL], ["perc", "perc"])
    y_train = utils.generate_y_cat(train_df.CLOSE)
    X_train = X_train.dropna(axis=0)
    y_train = y_train.dropna(axis=0)

    # get intersection of all dataframes
    common_index = X_train.index.intersection(y_train.index)
    X_train = X_train[X_train.index.isin(common_index)].to_numpy()
    y_train = y_train[y_train.index.isin(common_index)].to_numpy(np.dtype(int))
    
    model = SVC(C=1.5, kernel="rbf", gamma="scale", class_weight="balanced", probability=True)
    fitted = model.fit(X_train, y_train)
    save = models.categorical.SVMWrapper(model=fitted)
    
    with open(f'saved_models/categorical/{txt_path}.p', 'wb') as f:
        pickle.dump(save, f)

# Get SMA Accuracy and Opp Cost

In [ ]:
futuresList = utils.futuresList

start_date = date(2000, 1, 1)

# retrieve parameter grid
pbar = ProgressBar()

acc = []
opp_cost = []
for future in pbar(futuresList):
    print(future)
    # load data
    df = pd.read_csv(f"tickerData/{future}.txt", parse_dates = ["DATE"])
    df.columns = ['DATE', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL', 'OI', 'P', 'R', 'RINFO']
    df = df.set_index("DATE")
    df = df[(df.VOL != 0) & (df.CLOSE != 0)]
    df = df.dropna(axis=0)
    
    # load X and y
    X_df = utils.generate_X_df([df.CLOSE, df.VOL], ["perc", "perc"])
    y_df = utils.generate_y_cat(df.CLOSE)
    cost_df = utils.perc_change(df.CLOSE, shift=0)

    # run walk forward validation 
    model = SVC(C=1.5, kernel="rbf", gamma="scale", class_weight="balanced", probability=True)
    win_results, agg_results = models.categorical.walk_forward(
        model = model, X = X_df, y = y_df, cost_weight = cost_df, rolling = True, 
        max_windows = 100, start_index = start_date
    )
    acc.append(agg_results.loc["SMA", "accuracy"])
    opp_cost.append(agg_results.loc["SMA", "opp_cost"])
    
print("avg accuracy", np.mean(acc))
print("avg opp cost", np.mean(opp_cost))

In [ ]:
import numpy as np
print("avg accuracy", np.mean(acc))
print("avg opp cost", np.mean(opp_cost))